In [23]:
import torch
import brt.nn as nn
from brt.router import RandomScatterRouter, GatherRouter


class MoE(nn.Module):
    def __init__(self):
        super().__init__()
        self.scatter_router = RandomScatterRouter(route_num=2)
        self.expert1 = nn.Linear(10, 10)
        self.expert2 = nn.Linear(10, 10)
        self.gather_router = GatherRouter(route_num=2)

    def forward(self, x):
        route_results, route_tags, loads = self.scatter_router(x)
        print(route_tags)
        x_0 = self.expert1(route_results[0])
        x_1 = self.expert2(route_results[1])
        x = self.gather_router([x_0, x_1], route_tags, loads)
        return x


class ThorMoE(nn.Module):
    def __init__(
        self,
    ):
        super().__init__()
        self.moe = MoE()

    def forward(self, x):
        return self.moe(x)


thor_moe = ThorMoE()
x = torch.arange(0, 30, dtype=torch.float32).view(3, 10)
x = thor_moe(x)
print(x)


[tensor([[0],
        [1]]), tensor([[2]])]
tensor([[ -6.1178,  -5.4062,   0.1572,   1.1319,   0.0743,   2.5648,   3.2278,
          -1.2482,   1.8481,   2.0728],
        [-11.5727, -16.8110,  -1.7224,   5.1504,   0.7890,   4.1539,   7.6824,
          -5.6100,   5.2367,   5.3597],
        [ -1.4000,  -6.7778,   1.9196,   3.4563, -12.3829,  -4.1050,  18.9692,
         -13.0372,  -6.0614,  -2.5030]], grad_fn=<ScatterAddBackward0>)
